In [1]:
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import (
    LoraConfig,
    PeftModel,
    prepare_model_for_kbit_training,
    get_peft_model,
)
import os, torch, wandb
from datasets import load_dataset
from trl import SFTTrainer, setup_chat_format

import pandas as pd
from datasets import Dataset

import bitsandbytes as bnb

from huggingface_hub import login

torch.cuda.empty_cache()

In [2]:
torch_dtype = torch.float16
attn_implementation = "eager"

# QLoRA config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch_dtype,
    bnb_4bit_use_double_quant=True,
)

base_model = "meta-llama/Llama-3.2-1B-Instruct"

# Load model
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    quantization_config=bnb_config,
    device_map="auto",
    attn_implementation=attn_implementation
)

# Load tokenizer
tokenizer= AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)
tokenizer.add_special_tokens({'pad_token': '[PAD]'})

1

In [3]:
data = pd.read_json("hf://datasets/Amod/mental_health_counseling_conversations/combined_dataset.json", lines=True)
data.head()


,Context,Response
0,I'm going through some things with my feelings...,"If everyone thinks you're worthless, then mayb..."
1,I'm going through some things with my feelings...,"Hello, and thank you for your question and see..."
2,I'm going through some things with my feelings...,First thing I'd suggest is getting the sleep y...
3,I'm going through some things with my feelings...,Therapy is essential for those that are feelin...
4,I'm going through some things with my feelings...,I first want to let you know that you are not ...


In [4]:
data['Context_length'] = data['Context'].apply(len)
filtered_data = data[data['Context_length'] <= 1500]
ln_Response = filtered_data['Response'].apply(len)
filtered_data = filtered_data[ln_Response <= 4000]

In [5]:
data_prompt = """Analyze the provided text from a mental health perspective. Identify any indicators of emotional distress, coping mechanisms, or psychological well-being. Highlight any potential concerns or positive aspects related to mental health, and provide a brief explanation for each observation.

### Input:
{}

### Output:
{}"""

EOS_TOKEN = tokenizer.eos_token
def formatting_prompt(examples):
    inputs       = examples["Context"]
    outputs      = examples["Response"]
    texts = []
    for input_, output in zip(inputs, outputs):
        text = data_prompt.format(input_, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }

In [6]:
training_data = Dataset.from_pandas(filtered_data)
training_data = training_data.map(formatting_prompt, batched=True)

Map:   0%|          | 0/3480 [00:00<?, ? examples/s]

In [7]:

def find_all_linear_names(model):
    cls = bnb.nn.Linear4bit
    lora_module_names = set()
    for name, module in model.named_modules():
        if isinstance(module, cls):
            names = name.split('.')
            lora_module_names.add(names[0] if len(names) == 1 else names[-1])
    if 'lm_head' in lora_module_names:  # needed for 16 bit
        lora_module_names.remove('lm_head')
    return list(lora_module_names)

modules = find_all_linear_names(model)


# LoRA config
peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=modules
)

new_model = "llama-3.2-3b-it-Ecommerce-ChatBot"

#Hyperparamter
training_arguments = TrainingArguments(
    output_dir=new_model,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=2,
    optim="paged_adamw_32bit",
    num_train_epochs=1,
    eval_strategy="steps",
    eval_steps=0.2,
    logging_steps=1,
    warmup_steps=10,
    logging_strategy="steps",
    learning_rate=2e-4,
    fp16=False,
    bf16=False,
    group_by_length=True,
    report_to="wandb"
)

# Create train/test split
full_dataset = training_data.train_test_split(test_size=0.1, seed=42)

# Setting sft parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=full_dataset["train"],
    eval_dataset=full_dataset["test"],
    peft_config=peft_config,
    max_seq_length=512,
    dataset_text_field="text",
    tokenizer=tokenizer,
    args=training_arguments,
    packing=False,
)


/home/simon/.local/lib/python3.10/site-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': max_seq_length, dataset_text_field. Will not be supported from version '0.13.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/home/simon/.local/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:300: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/home/simon/.local/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:328: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(


Map:   0%|          | 0/3132 [00:00<?, ? examples/s]

Map:   0%|          | 0/348 [00:00<?, ? examples/s]

In [8]:
text="I'm going through some things with my feelings and myself. I barely sleep and I do nothing but think about how I'm worthless and how I shouldn't be here. I've never tried or contemplated suicide. I've always wanted to fix my issues, but I never get around to it. How can I change my feeling of being worthless to everyone?"

inputs = tokenizer([
    data_prompt.format(text, "")
], return_tensors='pt', padding=True, truncation=True).to("cuda")


outputs = model.generate(**inputs, max_new_tokens = 2020, use_cache = True)

answer=tokenizer.batch_decode(outputs)
answer

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


['<|begin_of_text|>Analyze the provided text from a mental health perspective. Identify any indicators of emotional distress, coping mechanisms, or psychological well-being. Highlight any potential concerns or positive aspects related to mental health, and provide a brief explanation for each observation.\n\n### Input:\nI\'m going through some things with my feelings and myself. I barely sleep and I do nothing but think about how I\'m worthless and how I shouldn\'t be here. I\'ve never tried or contemplated suicide. I\'ve always wanted to fix my issues, but I never get around to it. How can I change my feeling of being worthless to everyone?\n\n### Output:\n**Emotional Distress:**\n- **Low self-esteem:** The individual\'s self-perception is marked by feelings of worthlessness and inadequacy, which are significant indicators of emotional distress.\n- **Anxiety:** The constant worry about being worthless suggests anxiety, which is a common symptom of depression and other mental health di

In [9]:
answer_short = answer[0].split("### Output:")[-1]
print("Answer of the question is:", answer_short)

Answer of the question is: 
**Emotional Distress:**
- **Low self-esteem:** The individual's self-perception is marked by feelings of worthlessness and inadequacy, which are significant indicators of emotional distress.
- **Anxiety:** The constant worry about being worthless suggests anxiety, which is a common symptom of depression and other mental health disorders.
- **Depression:** The individual's feelings of worthlessness and inaction suggest depression, a condition characterized by persistent feelings of sadness or hopelessness.

**Coping Mechanisms:**
- **Self-destructive behaviors:** The individual mentions engaging in self-destructive behaviors, such as not sleeping and doing nothing but thinking about their feelings, which may be a coping mechanism to avoid or escape from their emotional distress.
- **Avoidance:** The individual avoids tasks and activities that might be perceived as "fixing" their issues, which could be a coping mechanism to avoid dealing with their emotions.



In [10]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: simonroy99 (simonroy99-cole-de-technologie-sup-rieure). Use `wandb login --relogin` to force relogin


  0%|          | 0/1566 [00:00<?, ?it/s]

{'loss': 2.77, 'grad_norm': 1.2635055780410767, 'learning_rate': 2e-05, 'epoch': 0.0}
{'loss': 2.7064, 'grad_norm': 1.186571717262268, 'learning_rate': 4e-05, 'epoch': 0.0}
{'loss': 2.6669, 'grad_norm': 1.224671721458435, 'learning_rate': 6e-05, 'epoch': 0.0}
{'loss': 2.8356, 'grad_norm': 1.1687082052230835, 'learning_rate': 8e-05, 'epoch': 0.0}
{'loss': 2.6183, 'grad_norm': 1.2120256423950195, 'learning_rate': 0.0001, 'epoch': 0.0}
{'loss': 2.6172, 'grad_norm': 1.1181726455688477, 'learning_rate': 0.00012, 'epoch': 0.0}
{'loss': 2.8756, 'grad_norm': 1.155635952949524, 'learning_rate': 0.00014, 'epoch': 0.0}
{'loss': 2.7535, 'grad_norm': 1.1533392667770386, 'learning_rate': 0.00016, 'epoch': 0.01}
{'loss': 2.6182, 'grad_norm': 1.1867891550064087, 'learning_rate': 0.00018, 'epoch': 0.01}
{'loss': 2.4406, 'grad_norm': 1.3232975006103516, 'learning_rate': 0.0002, 'epoch': 0.01}
{'loss': 2.8411, 'grad_norm': 1.4021612405776978, 'learning_rate': 0.00019987146529562984, 'epoch': 0.01}
{'loss

Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.


{'loss': 2.2059, 'grad_norm': 1.4062678813934326, 'learning_rate': 0.0001609254498714653, 'epoch': 0.2}


...

Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.


{'loss': 2.1361, 'grad_norm': 1.4099690914154053, 'learning_rate': 3.9845758354755785e-05, 'epoch': 0.8}


  0%|          | 0/348 [00:00<?, ?it/s]

{'eval_loss': 1.7811052799224854, 'eval_runtime': 33.2244, 'eval_samples_per_second': 10.474, 'eval_steps_per_second': 10.474, 'epoch': 0.8}
{'loss': 2.1911, 'grad_norm': 1.5782781839370728, 'learning_rate': 3.9717223650385605e-05, 'epoch': 0.8}
{'loss': 2.1361, 'grad_norm': 1.6884671449661255, 'learning_rate': 3.958868894601543e-05, 'epoch': 0.8}
{'loss': 2.2349, 'grad_norm': 2.1563656330108643, 'learning_rate': 3.9460154241645245e-05, 'epoch': 0.8}
{'loss': 1.9097, 'grad_norm': 1.608792781829834, 'learning_rate': 3.9331619537275065e-05, 'epoch': 0.8}
{'loss': 1.7614, 'grad_norm': 1.532404899597168, 'learning_rate': 3.9203084832904885e-05, 'epoch': 0.81}
{'loss': 1.7043, 'grad_norm': 3.3838300704956055, 'learning_rate': 3.9074550128534705e-05, 'epoch': 0.81}
{'loss': 2.2757, 'grad_norm': 1.6535460948944092, 'learning_rate': 3.8946015424164526e-05, 'epoch': 0.81}
{'loss': 2.1054, 'grad_norm': 1.7962642908096313, 'learning_rate': 3.8817480719794346e-05, 'epoch': 0.81}
{'loss': 2.0516, '

TrainOutput(global_step=1566, training_loss=1.8647156544267598, metrics={'train_runtime': 775.6868, 'train_samples_per_second': 4.038, 'train_steps_per_second': 2.019, 'total_flos': 5749160402288640.0, 'train_loss': 1.8647156544267598, 'epoch': 1.0})

In [11]:
wandb.finish()

eval/loss,█▅▂▁
eval/runtime,▁▁█▂
eval/samples_per_second,██▁▇
eval/steps_per_second,██▁▇
train/epoch,▁▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▇▇▇▇▇▇▇█
train/global_step,▁▁▁▂▂▃▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇▇▇█
train/grad_norm,▁▃▁▂▃▁▁▂▃▂▄▃▅█▄▃▃▃▂▃▇▂▄▆▄▄▄▃▃▅▄▃▅▃▄▄▇█▃▁
train/learning_rate,▁████▇▇▇▆▆▆▆▅▅▅▅▅▅▅▄▄▄▄▄▄▄▄▃▃▃▃▃▂▂▂▂▁▁▁▁
train/loss,█▆▆▆█▆▅▆▅▇▄▅▄▁▅▄▆▄▅▅▅▁▄▅▅▅▃▁▅▂▂▂▆▄▂▃▃▄▆▃
eval/loss,1.78111
eval/runtime,33.2244


In [12]:
text="I'm going through some things with my feelings and myself. I barely sleep and I do nothing but think about how I'm worthless and how I shouldn't be here. I've never tried or contemplated suicide. I've always wanted to fix my issues, but I never get around to it. How can I change my feeling of being worthless to everyone?"

inputs = tokenizer([
    data_prompt.format(text, "")
], return_tensors='pt', padding=True, truncation=True).to("cuda")


outputs = model.generate(**inputs, max_new_tokens = 2020, use_cache = True)

answer=tokenizer.batch_decode(outputs)
answer_short = answer[0].split("### Output:")[-1]
print("Answer of the question is:", answer_short)

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Answer of the question is: 
It's hard to say how old you are, but I think I know what you're talking about. I have been there for a long time, but I'm glad you're reaching out. I am not sure where you live, but I think it is important to find a counselor who can help you with your feelings. I have been working with a wonderful counselor in my area, and she has helped many people who have been struggling with suicidal thoughts. I am sure she can help you as well. I encourage you to find a counselor and make an appointment. <|eot_id|>
